# [CPSC 222](https://github.com/GonzagaCPSC222) Intro to Data Science
[Gonzaga University](https://www.gonzaga.edu/)

[Gina Sprint](http://cs.gonzaga.edu/faculty/sprint/)
## DA5 Jupyter Notebook (100 pts)

## Learner Objectives
At the conclusion of this programming assignment, participants should be able to:
* Use Matplotlib to visualize data
* Write Markdown and code cells in Jupyter Notebook
* Typeset equations using Latex
* Tell a data science story
 
## Prerequisites
Before starting this programming assignment, participants should be able to:
* Use pandas for data analysis
* Clean data

## Acknowledgments
Content used in this assignment is based upon information in the following sources:
* [Centers for Medicaid and Medicare Services IRF-PAI training manual](https://www.cms.gov/medicare/medicare-fee-for-service-payment/inpatientrehabfacpps/downloads/irfpai-manual-2012.pdf)

## Github Classroom Setup
For this assignment, you will use GitHub Classroom to create a private code repositories to track code changes and submit your assignment. Open this DA5 link to accept the assignment and create a private repository for your assignment in Github classroom: https://classroom.github.com/a/UO7Y1WAo

Your repo, for example, will be named GonzagaCPSC222/da5-yourusername (where yourusername is your Github username). I highly recommend committing/pushing regularly so your work is always backed up. We will grade your most recent commit, even if that commit is after the due date (your work will be marked late if this is the case).

## Programming (65 pts)
For this assignment, we are going to tell a story about exploratory data analysis (EDA) of a dataset. Your source code, explanations/log, and charts will be combined in a Jupyter Notebook, PatientEDA.ipynb, with well-organized, interleaved code cells and markdown cells. Here are general requirements that your Notebook should conform to:
* Your EDA story should be logically divided into different section levels, appropriately labeled using markdown headers, and contain well-written commentary describing the code, results, and insights you come up with
* Each chart that you generate must be generated inline in the Notebook and include a figure title and labels on the x and y axes where appropriate
* Each formula that you use in your code must be typeset using Latex and described in markdown (this includes formulas used for stats, even if those stats are implemented in a pandas function/method you call)
* Reusable utility code should be defined in functions in a `utils.py` file
    * Import `utils.py` in your Notebook
    * Call functions defined in `utils.py` to keep your Notebook code cells short
    * See the [coding standard](https://nbviewer.jupyter.org/github/GonzagaCPSC222/DAs/blob/master/Coding%20Standard.ipynb) for more details on Jupyter Notebook best practices

We are going to take a look at a real-world example of clinical data that needs cleaning and contains interesting insights once aggregated and visualized. We are going to work with this data in the following ways:
1. Load the data 
1. Clean the data
1. Aggregate the data and compute summary statistics
1. Visualize the data

Note: these would be good headers to have (at least) in your Notebook :)

### Load the Data
Download the patient_data_to_clean.csv from the DAs repo on Github: https://github.com/GonzagaCPSC222/DAs/blob/master/files. One way to download a file is to click "Raw" then right click on the page and click "Save As." Move this file into the same folder as your local DA5 Git repo. This dataset contains gender, marital status, and rehabilitation impairment category (RIC) information from 4,555 inpatient rehabilitation patients. The data has been de-identified and randomized. Here is a sample of the format of the data in the csv file:

|ID|Gender|Age|Marital Status|RIC|Admission Total FIM Score|Discharge Total FIM Score|
|-|-|-|-|-|-|-|
|0|M|80|Widowed|8|40|89|
|1|M|90|Divorced|1|65|75|
|2|M|53|Married|2|67|99|
|...|...|...|...|...|...|...|

And a description of each attribute:
* ID (integer): Index of the dataset. Counting numbers starting at 0.
* Gender (string): Gender of the patient, "M" for male and "F" for female.
* Age (integer): Age of the patient in years
* Marital Status (string): Description of the patient's marital status. No coding system enforced.
* RIC (integer): RIC of the patient assigned according to Appendix B in the [Centers for Medicaid and Medicare Services IRF-PAI training manual](https://www.cms.gov/medicare/medicare-fee-for-service-payment/inpatientrehabfacpps/downloads/irfpai-manual-2012.pdf).
* Admission Total FIM Score: The admission total Functional Independence Measure (FIM) score of the patient. 
    * The FIM is a clinical assessment used to measure patient functioning at inpatient rehabilitation hospitals. The FIM is measured at two distinct points in time: admission and discharge. 
    * The FIM measures the level of assistance required to perform 18 activities of daily living (ADL) tasks (e.g. eating, walking, problem-solving, etc.). 
    * The tasks are categorized as either motor (13 tasks) or cognitive (5 tasks). Each task is scored on a 7-point ordinal scale to measure independence as determined by the amount of assistance required to perform each ADL task. 
    * For more information about the FIM, see Section III in the [Centers for Medicaid and Medicare Services IRF-PAI training manual](https://www.cms.gov/medicare/medicare-fee-for-service-payment/inpatientrehabfacpps/downloads/irfpai-manual-2012.pdf).
* Discharge Total FIM Score: The discharge total FIM score of the patient.

Note: it would be good to have these attribute descriptions in your Notebook :)

Read the patient data into a Pandas `DataFrame` object. The index column is the ID column (column 0). The header row is the first row in the file.

### Clean the Data
Let's take a look at each column in the data and how the data needs to be cleaned:
* ID: No cleaning necessary
* Gender: No cleaning necessary
* Age: No cleaning necessary
* Marital Status: This column is quite messy compared to the other columns. If we only look at the first 8 rows of the dataset, the Marital Status column looks like it is well coded; however, we see for ID 8 there is a period after "Married." which doesn't match any of the previous "Married" entries. Upon further exploration, we see this column truly was free response for the clinicians to enter text. For example, take a look at IDs 33, 36, 38, 41, and 42! We are going to after do some string matching to apply a uniform encoding for the Marital Status column. When cleaning this column, use a simple rule based system to handle the various spellings and word choices that represent the following marital statuses: "Never married", "Divorced", "Married", "Widowed", "Separated"
    * Note: Try your best to clean each martial status value (this does require manual inspection). Document/log any entries that you feel you don't have sufficient information for to assign one of the above labels. For these cases, overwrite the entry with a null value (`NaN`) to represent missing data. 
* RIC (integer): Decode the integer RIC label to the plain text string RIC label. Here is a dictionary storing the integer-string mappings: `ric_decoder = {1: "Stroke", 2: "TBI", 3: "NTBI", 4: "TSCI", 5: "NTSCI", 6: "Neuro", 7: "FracLE", 8: "ReplLE", 9: "Ortho", 10: "AMPLE", 11: "AMP-NLE", 12: "OsteoA", 13: "RheumA", 14: "Cardiac", 15: "Pulmonary", 16: "Pain", 17: "MMT-NBSCI", 18: "MMT-BSCI", 19: "GB", 20: "Misc", 21: "Burns"}`
    1. "Stroke"
    1. "TBI" (Traumatic brain injury)
    1. "NTBI" (Non-traumatic brain injury)
    1. "TSCI" (Traumatic spinal cord injury)
    1. "NTSCI" (Non-traumatic spinal cord injury)
    1. "Neuro" (Neurologic conditions)
    1. "FracLE" (Fracture, lower extremity)
    1. "ReplLE" (Joint replacement, lower extremity)
    1. "Ortho" (Other orthopaedic)
    1. "AMPLE" (Amputation, lower extremity)
    1. "AMP-NLE"(Amputation, upper extremity or other)
    1. "OsteoA" (Osteoarthritis)
    1. "RheumA" (Rheumatoid arthritis)
    1. "Cardiac" (Cardiac disorders)
    1. "Pulmonary" (Pulmonary disorders)
    1. "Pain" (Pain syndromes)
    1. "MMT-NBSCI" (Major multiple trauma, non brain injury or spinal cord injury)
    1. "MMT-BSCI" (Major multiple trauma, brain injury or spinal cord injury)
    1. "GB": (Guillain-Barre Syndrome)
    1. "Misc" (Miscellaneous)
    1. "Burns"
* Admission Total FIM Score: No cleaning necessary
* Discharge Total FIM Score: No cleaning necessary

Write the cleaned Pandas `DataFrame` out to a new file called patient_data_cleaned.csv (include this file in your Git repo). This dataset is now cleaned and ready for use in the next step of our data analysis pipeline.

### Aggregate the Data
Construct a Pandas `Series` with the following statistics about the cleaned data:
1. `patients_total`: total number of patients
1. `males_total`: total number of males
1. `females_total`: total number of females
1. `married_total`: total number of married patients
1. `most_common_RIC`: RIC label for the most commonly occurring RIC
1. `most_common_RIC_total`: total number of patients with the most commonly occurring RIC
1. `stroke_age_avg`: average age for stroke patients
1. `stroke_age_std`: standard deviation of age for stroke patients
1. `stroke_age_male_avg`: average age for male stroke patients
1. `stroke_age_male_std`: standard deviation of age for male stroke patients
1. `stroke_age_female_avg`: average age for female stroke patients
1. `stroke_age_female_std`: standard deviation of age for female stroke patients

Display this `Series` and document/log your observations.

### Visualize the Data
For each RIC with enough data, produce the following charts (histograms and scatter plots) using Matplotlib.

#### Histograms
Age [histogram](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.pyplot.hist.html)
* X axis label: "Age (years)"
* Y axis label: "Frequency"
* Title: "`<RIC>` Age (N=`<total>`): Mean: `<2 decimal places>`, StdDev: `<2 decimal places>`"
* Bars: Green, 30 bins
* Example:   

<img src="https://github.com/GonzagaCPSC222/DAs/raw/master/figures/ReplLE_age.png" width="400">

Display the charts and document/log your observations.

#### Scatter Plots
FIM [scatter](http://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.scatter.html) plot
* X axis label: Admission FIM score
* Y axis label: Discharge FIM score
* Title: "`<RIC>` (N=`<total>`)"
* Male scatter points: Blue, circle markers ("."), size 100, label "Male (N=`<total>`)"
* Female scatter points: Red, plus markers ("+"), size 100, label "Female (N=`<total>`)" 
* Y = X [line](https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.plot): Black, dashed line style ("--"), [x limits](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.set_xlim.html) and [y limits](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.set_ylim.html) are [0, 140]
    * This is called a "no change" line, Y = X. This line represents when the discharge FIM score is the same as the admission FIM score. Patients above this line showed a FIM score improvement, patients below this line showed a regression.
* [Legend](https://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.legend): lower right corner ("4")
* Example:  

<img src="https://github.com/GonzagaCPSC222/DAs/raw/master/figures/ReplLE_fim.png" width="500"> 

Display the charts and document/log your observations.

### Bonus (5 pts)
Add the following improvements to the age histogram plots for each RIC category:
* Use Latex to show the plot title a follows: "`<RIC>` Age (N=`<total>`): $\mu=$ `<2 decimal places>`, $\sigma=$ `<2 decimal places>`"
* Use SciPy's [Normal Probability Density Function (PDF)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html) to show an estimated normal curve fit over the histogram: Red, line width 3

Example: 

<img src="https://github.com/GonzagaCPSC222/DAs/raw/master/figures/ReplLE_age_BONUS.png" width="400">

## Project Part 5 (15 pts)
For the dataset you are most likely going to use for your project (note: you don't need to commit to this now, but soon!!), prepare a CSV file of your data and include it in your Git repo. In a Jupyter Notebook named ProjectPart5.ipynb, describe the dataset, load the dataset into a Pandas `DataFrame`, and perform any cleaning the dataset requires. Document any issues or questions you have cleaning your data. Now, produce at least one plot in your Notebook. The plot should contain at least two columns of data and should be a different plot type than a histogram or line/scatterplot. Feel free to use a different plot type covered in class or try a new one by exploring the [Matplotlib examples](https://matplotlib.org/3.3.2/tutorials/introductory/sample_plots.html). 

## Data Ethics (15 pts)
Read [The 70% problem: Hard truths about AI-assisted coding](https://addyo.substack.com/p/the-70-problem-hard-truths-about). This is a blog post from December 2024 that captures the essence of a tangent I want on recently in class. If you recall, I was emphasizing that experts in a field can easily detect AI-generated work, especially when the "author" uses it as-is without adding human-value to it. Relating this specifically to software development and data science, this would be like when someone tries to use AI-generated code without understanding it, adapting it to requirements, and testing it thoroughly. The example I gave in class was from last semester when a team of students submitted a Jupyter Notebook project report that included "This project was completed as part of CPSC 322 at the University of British Columbia, Fall 2024" and machine learning code that looked just like the popular, open-source machine learning library [sci-kit learn](https://scikit-learn.org/stable/) ([source code here](https://github.com/scikit-learn/scikit-learn) if you want to poke around and see how it looks very different from the code we write in class).

Please read the article directly and resist the urge to use AI to summarize it. Also, while I don't mind if you use AI to work with you to brainstorm ideas, **please do not outsource your thinking and writing to AI**. If you do use AI in any capacity, you must cite your use of it.

In a Jupyter Notebook called Ethics.ipynb, provide your reflection on the following discussion points:
1. Re-read each of the bullet points under the "What does this mean for you?" section. Reflect on how you use or do not use AI-assisted coding for these tasks. What room for improvement do you have?
1. The article talks about how learning to program requires "patience and dedication" and how "the goal isn't to write more code faster"; however, this is exactly the opposite of what many people hope to achieve by using AI tools in the first place. When you are problem solving, programming, and debugging, how patient and dedicated are you to the task at hand? Do you find yourself often wanting to just "get it done" and move on, getting frustrated when it doesn't go as smoothly as you want? Please share the positive and negative self-talk/internal dialogue you experience when faced with coding challenges.
1. The article frequently discusses "production-ready" code and provides several examples of when AI-generated code alone fails to produce production-ready code. As you are starting to branch out from learning the "basics of programming" to "what cool apps can I build?", are you starting to think beyond just trying to get code to "work"? If so, what are you becoming more conscientious of as you code? If not, after reading the article, what do you think you could focus on more as you code to start thinking about increasing the quality of your work and making your code closer to production-ready?
1. There were alot of gems in this article that I just love. What are a few gems from the article you particularly appreciated/resonated with and why? For example, here are a few from me:
    1. "The return of software as craft? While AI has made it easier than ever to build software quickly, we're at risk of losing something crucial - the art of creating truly polished, consumer-quality experiences."
        * This resonated with me because I'm a firm believer in the craftsmen mindset -- wanting to become great at what you do and produce quality work.
    1. "AI isn't making our software dramatically better because software quality was (perhaps) never primarily limited by coding speed. The hard parts of software development – understanding requirements, designing maintainable systems, handling edge cases, ensuring security and performance – still require human judgment."
        * I love the author's enumeration of the "hard parts." Oftentimes we think of software development/engineering as strictly code writing. While coding is a large part of it, it is not the only part we need to be proficient at.

Note: if you are interested in what is meant by "P2 bugs", take a look at this blog post [How to Prioritize Your Bugs: a 2024 Guide](https://fibery.io/blog/product-management/bug-prioritization/)

Your reflection write-up should be written using full sentences and should be grammaticallly correct. Proof read your writing before you submit it!!

## Submitting Assignments
1. Turn in your assignment files via a Github Classroom repo. See the "Github Classroom Setup" section at the beginning of this document for details on how to do this.
    1. Your repo should contain all of the files needed to run and test your solution (e.g. .py file(s), input files, etc.).
    1. Double-check that this is the case by "pretending to be the grader": clone (or download a zip) your submission repo and run your code like when we grade it.
1. Submit this DA’s associated assignment in Canvas to mark your DA as "done" and ready for grading. We will then pull your Github repo and grade your DA as soon as possible. The date and time you submit the DA assignment in Canvas will be used for marking your assignment as "late" or "on-time."

## Grading Guidelines
This assignment is worth 100 points + 5 points bonus. Your assignment will be evaluated based on a successful execution in Jupyter Lab (using the Anaconda Python Distribution v3.12) and adherence to the program requirements. We will grade according to the following criteria:
* 5 pts for decoding the RIC
* 10 pts for cleaning the marital status and documenting assigned `NaN`s
* 5 pts for writing the cleaned DataFrame to a CSV file
* 10 pts for computing the correct stats, storing them in a Series, and displaying the Series
* 10 pts for correct RIC age histogram plots
* 10 pts for correct RIC FIM scatterplots
* 8 pts for data storytelling using Markdown cells
* 7 pts for typesetting stats formulas using Latex
* 5 pts for adherence to course [coding standard](https://nbviewer.jupyter.org/github/GonzagaCPSC222/DAs/blob/master/Coding%20Standard.ipynb)
* 15 pts for including a CSV and code to load the data and plot it in ProjectPart5.ipynb
* 15 pts for quality, clarity, and creativity in the data ethics write-up